In [1]:
!pip install sqlalchemy

In [2]:
import sqlalchemy as db
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Text, ForeignKey
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker

Base = declarative_base()
engine = db.create_engine('sqlite:///clother_category.db')
connection = engine.connect()
Session = sessionmaker(bind=engine)

In [3]:
class Category(Base):
    __tablename__ = 'category'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('category.id'), nullable=True)
    title = Column(Text, nullable=False)
    last_level = Column(Integer, default=1, nullable=False)

    subcategories = relationship('Category',
                                    backref=backref('parent', remote_side=[id]),
                                    lazy=True)

In [4]:
Base.metadata.create_all(engine)

In [5]:
session = Session()

categories = json.load(open("./categories.json", 'r'))
for entry in categories:
    category = Category(parent_id=entry['parent_id'], title=entry['title'])

    session.add(category)
    session.commit()

delete_query = Category.__table__.delete().where(Category.title.ilike("view all"))
session.execute(delete_query)
session.commit()

categories = session.query(Category).all()
for category in categories:
    if len(category.subcategories) > 0:
        category.last_level = False
        session.commit()

In [7]:
[x.last_level for x in session.query(Category).first().subcategories]

[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [8]:
[len(x.subcategories) for x in session.query(Category).first().subcategories]

[10, 5, 7, 4, 6, 4, 4, 0, 0, 8, 2, 9, 2, 9, 7, 3, 9, 13, 3, 6, 3, 14, 13, 0]